In [2]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os
import sqlite3


/home/qiyue_ren/anaconda3/envs/mfe-preprogram/lib/python3.8/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [3]:
connection_string = 'sqlite:///data/data.db'
events_db = 'crypto_events_database'

In [4]:
with sqlite3.connect('data/data.db') as conn:
    cur = conn.cursor()
    cur.execute(f"SELECT ts, token FROM ohlc")
    data = cur.fetchall()
    data = pd.DataFrame(data, columns = ['ts','token'])
    data['ts'] = pd.to_datetime(data['ts'])
    
    tokens = data.token.unique().tolist()
    
    for i in range(data.index[-1], 0, -1):
        ts = data.ts[i]
        if ts.weekday() + 1 == 1:
            end_date = ts.date()
            start_date = end_date - timedelta(days=7 * 2)
            break

In [5]:
tokens

['BTC',
 'ETH',
 'USDT',
 'SOL',
 'ADA',
 'DOT',
 'AVAX',
 'ATOM',
 'CRV',
 'AAVE',
 'COMP']

In [6]:
run_dates = [pd.Timestamp(start_date)]
run_dates

[Timestamp('2021-11-29 00:00:00')]

In [7]:
# create a output folder
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)
base_output_folder

'outputs'

In [8]:
for run_date in run_dates:
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=7*2)}', end='\r')
        res = pm.execute_notebook(
            'hw4_template.ipynb',
            f'{base_output_folder}/hw4_market_analysis_{token}_{run_date}.ipynb', # outputing
            parameters = {
                'from_date': f'{run_date}',
                'to_date': f'{run_date + timedelta(days=7*2)}',
                'token': token,
                'connection_string': connection_string,
                'events_db': events_db
            }
        )

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

In [9]:
# a list of notebooks  
nbs = sb.read_notebooks('outputs')
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'returns': scrap['prices'].data.returns.mean(),
        'volatility': scrap['prices'].data.vol.mean(),
        'maximum drawdown': scrap['prices'].data.max_drawdown[-1],
        'close price range':scrap['prices'].data.close.iloc[-1]-scrap['prices'].data.close.iloc[0] #range of last close price - first close price
    }) 
    for book, scrap in nbs.notebook_scraps.items()
])

,token,from_date,to_date,returns,volatility,maximum drawdown,close price range
0,AAVE,2021-11-29 00:00:00,2021-12-13 00:00:00,-7.697463e-04,0.230765,-0.004353,-58.6450
1,ADA,2021-11-29 00:00:00,2021-12-13 00:00:00,-4.241618e-04,0.241399,-0.004397,-0.2445
2,ATOM,2021-11-29 00:00:00,2021-12-13 00:00:00,-1.776982e-04,0.333404,-0.005530,-2.7560
3,AVAX,2021-11-29 00:00:00,2021-12-13 00:00:00,-5.540979e-04,0.319050,-0.005600,-22.9300
4,BTC,2021-11-29 00:00:00,2021-12-13 00:00:00,-3.569235e-04,0.166549,-0.003132,-6999.8400
5,COMP,2021-11-29 00:00:00,2021-12-13 00:00:00,-1.015116e-03,0.242214,-0.003601,-88.2800
6,CRV,2021-11-29 00:00:00,2021-12-13 00:00:00,-5.385247e-04,0.364842,-0.008219,-1.0362
7,DOT,2021-11-29 00:00:00,2021-12-13 00:00:00,-4.540131e-04,0.271552,-0.005591,-6.0050
8,ETH,2021-11-29 00:00:00,2021-12-13 00:00:00,-6.065377e-05,0.192750,-0.004331,-141.7100
9,SOL,2021-11-29 00:00:00,2021-12-13 00:00:00,-3.446384e-04,0.270302,-0.005606,-26.6870
